# Automatic TICI

## Import modules

In [15]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from scipy.io import loadmat
from sklearn.model_selection import KFold
import skimage.filters as filters

## Select Feature Image

In [1]:
def gray2binary(image_set):
    thresholds = list(map(filters.threshold_li, image_set))
    # When an image's frequency is too low, filters.threshold_li may 
    # cause a divide-by-zero RuntimeWarning, and the corresponding
    # threshold will be 0. So we need to remove zeros in thresholds.
#     thresholds = list(filter(lambda th: th > 0, thresholds))
    # print(thresholds)
    
    min_threshold = min(thresholds)
    # print('min_threshold = ', min_threshold)
    result = []
    for i in range(len(image_set)):
        result.append(image_set[i] > thresholds[i])
    return result
#     return np.array(list(map(lambda img : img > min_threshold, image_set)))

In [2]:
def summarize_middle_artery(image_set):
    # Assume all the images in an image set have the same dimensions.
    image_count, image_height, image_width = np.shape(image_set)
    result = []
    for image in image_set:
        # Only consider the middle 1/4 of the upper 1/4 image.
        middle_section = image[0:int(image_height/4), int(image_width*3/8):int(image_width*5/8)]
        # plt.figure()
        # plt.imshow(middle_section, cmap='gray')
        
        summary = np.array(list(map(all, middle_section)))
        # print(np.count_nonzero(summary))

        # summary = np.tile(summary[:, np.newaxis], 200)
        result.append(summary)
    return np.array(result)

In [4]:
def choose_image(image_set):
    binary_image_set = gray2binary(image_set)
    middle_artery_summary_image_set = summarize_middle_artery(binary_image_set)
    image_count, image_height = np.shape(middle_artery_summary_image_set)
    
    flagOn = False
    previous_nonzeroCount = -1
    for i in range(image_count):
        nonzeroCount = np.count_nonzero(middle_artery_summary_image_set[i])
        if flagOn and nonzeroCount < image_height/2:
            return image_set[i]
        elif previous_nonzeroCount >= 0 and nonzeroCount - previous_nonzeroCount > image_height/2:
            flagOn = True
        previous_nonzeroCount = nonzeroCount
    return None

## Load data & extract images and TICI scores

In [12]:
# Get the path of the data directory.
data_dir = os.path.join(os.getcwd(), 'data')
# Get a list of full paths of all mat files in the data directory.
for root, _dirs, files in os.walk(data_dir):
    files = list(filter(lambda fname: fname.lower().endswith('.mat'), sorted(files)))
nfiles = len(files)
print('{} files found in the data directory \'{}\'.'.format(nfiles, data_dir))

201 files found in the data directory '/Users/ceciliawang/Desktop/UCLA/Senior Year/Spring 2019/CS 168/CS168-Automatic-TICI/data'.


In [13]:
# Given the data_dir and a file name, extract an image from
# the set of images and the TICI score.
# By default, nothing is printed.
# If verbose=1, print the keys of the mat file content (which is a dictionary).
# If verbose=2, print the 3 TICI scores for debugging purpose.
def extract_data_file(data_dir, fname, verbose=False):
    content = loadmat(os.path.join(data_dir, fname))
    
    if verbose == 1:
        print('{}\tkeys={}'.format(fname, sorted(content.keys())))
    if verbose == 2:
        print('{}\t\tTICI_Dr1={}\tTICI_Dr2={}\tTICI_report={}'.format(fname, content['TICI_Dr1'], content['TICI_Dr2'], content['TICI_report']))

    raw_image_set, TICI = content['X'], content['TICI_report']
    # Originally, raw_image_set[:, :, k] is the kth image.
    # Reorder the dimensions such that raw_image_set[k, :, :] is the kth image.
    image_set = np.transpose(raw_image_set, (2, 0, 1))
    # Only one image from each image set is selected to be fed into the model.
    # For simplicity, the image in the middle of each image set is selected just for now.
    count, _, __ = np.shape(image_set)
    image = choose_image(image_set)
    return image, TICI

In [16]:
# From the first mat file, learn the structure and the image set dimensions.
# Assume that all mat files have the same structure and that all the images 
# in each image set have the same dimensions, though each image sets may
# contain various number of images.
sample_image, sample_TICI = extract_data_file(data_dir, files[0], verbose=1)
image_shape = sample_image.shape
print(image_shape)

fractals_1.mat	keys=['TICI_Dr1', 'TICI_Dr2', 'TICI_report', 'X', '__globals__', '__header__', '__version__']
(1024, 1024)


In [35]:
# For debugging purpose, print the 3 TICI scores for the first 10 mat files.
for n in range(10):
    _, _ = extract_data_file(data_dir, files[n], verbose=2)

fractals_1.mat		TICI_Dr1=['2b']	TICI_Dr2=['2b']	TICI_report=['2a']
fractals_10.mat		TICI_Dr1=[[nan]]	TICI_Dr2=[[nan]]	TICI_report=['2a']
fractals_100.mat		TICI_Dr1=[[nan]]	TICI_Dr2=[[nan]]	TICI_report=[[3]]
fractals_101.mat		TICI_Dr1=[[nan]]	TICI_Dr2=[[nan]]	TICI_report=[[0]]
fractals_102.mat		TICI_Dr1=[[nan]]	TICI_Dr2=[[nan]]	TICI_report=['2b']
fractals_103.mat		TICI_Dr1=[[nan]]	TICI_Dr2=[[nan]]	TICI_report=[[nan]]
fractals_104.mat		TICI_Dr1=[[nan]]	TICI_Dr2=[[nan]]	TICI_report=['2b']
fractals_105.mat		TICI_Dr1=[[nan]]	TICI_Dr2=[[nan]]	TICI_report=[[0]]
fractals_106.mat		TICI_Dr1=[[nan]]	TICI_Dr2=[[nan]]	TICI_report=[[0]]
fractals_107.mat		TICI_Dr1=[[nan]]	TICI_Dr2=[[nan]]	TICI_report=['2a']


In [36]:
# Initialize a list for images and a numpy arrays for 
# corresponding TICI scores. Assume all images have 
# the same dimensions. The TICI score in each 
images = []
TICI_strings = []

# Extract image and TICI information for all mat files.
for n in range(nfiles):
    # Print the extracting progress.
    if n % 10 == 0:
        print('{} / {} done'.format(n, nfiles))
    image, TICI = extract_data_file(data_dir, files[n])
    images.append(image)
    # The TICI scores in the mat files are in the form of 
    # nested np.ndarray's of either strings, numbers, of nan.
    # e.g., ['2a'], [[3]], [[nan]]. With assumption of 
    # this structure, simplify TICI before append it to TICIs.
    while isinstance(TICI, np.ndarray):
        TICI = TICI[0] if len(TICI) > 0 else ''
    TICI_strings.append(str(TICI))

print(np.shape(images))
print(TICI_strings)

0 / 201 done
10 / 201 done
20 / 201 done
30 / 201 done
40 / 201 done
50 / 201 done
60 / 201 done
70 / 201 done
80 / 201 done
90 / 201 done
100 / 201 done
110 / 201 done
120 / 201 done
130 / 201 done
140 / 201 done
150 / 201 done
160 / 201 done
170 / 201 done
180 / 201 done
190 / 201 done
200 / 201 done
(201, 1024, 1024)
['2a', '2a', '3', '0', '2b', 'nan', '2b', '0', '0', '2a', '2b', '2a', '2a', '2b', '0', '2b', '2b', '0', 'nan', '3', 'nan', '2b', '1', '2b', '2a', '0', '2b', '2b', 'nan', '0', '2a', '0', '2a', '3', '2b', '2b', '2a', 'nan', '2b', 'nan', '0', '2a', '2a', '3', '2b', '2b', '0', '2b', '2b', '2b', '2a', '2a', '2b', '3', '2a', '2a', '2b', '2a', '2a', '2b', '2a', '2b', '2b', '0', '2a', '1', '2a', '2b', '3', '2a', '2a', '3', '0', '2a', '2a', '2b', '2b', '2a', '2a', '2b', '1', '2b', '3', '2a', '2b', '2a', '2b', '2b', '2a', '2a', 'nan', '0', '3', '2b', '0', '0', '0 (bilateral MCA)', '0', '0', '2b', '3', '3', '2b', '0', '2a', '2a', '0', '2a', '2b', '2a', '2b', '2b', '2b', '2b', '2b'

## Reformat TICI scores

In [37]:
# The number of different TICI scores.
# Including 0, 1, 2a, 2b, 3, nan.
num_TICI_classes = 6

# Convert a TICI string to a number
def map_TICI_str_to_num(TICI):
    relation = {
        '0': 0,
        '1': 1,
        '2a': 2,
        '2b': 3,
        '3': 4,
        'nan': 5,
        '0 (bilateral MCA)': 0,
        '2a?': 2
    }
    return relation[TICI]

# Convert a numerical encoded TICI to a string
def map_TICI_num_to_str(label):
    relation = ['0', '1', '2a', '2b', '3', 'nan']
    return relation[label]

# Convert TICI scores in the form of strings to numeric labels before fed to the model.
TICI_nums = list(map(map_TICI_str_to_num, TICI_strings))
print(TICI_nums)

# Convert the array of integer labels (0 ~ num_TICI_classes-1) to an array of 
# one-hot (aka one-of-K) encoded labels, for better accuracy.
TICI_one_hot = tf.keras.utils.to_categorical(TICI_nums, num_TICI_classes)
print(TICI_one_hot)

[2, 2, 4, 0, 3, 5, 3, 0, 0, 2, 3, 2, 2, 3, 0, 3, 3, 0, 5, 4, 5, 3, 1, 3, 2, 0, 3, 3, 5, 0, 2, 0, 2, 4, 3, 3, 2, 5, 3, 5, 0, 2, 2, 4, 3, 3, 0, 3, 3, 3, 2, 2, 3, 4, 2, 2, 3, 2, 2, 3, 2, 3, 3, 0, 2, 1, 2, 3, 4, 2, 2, 4, 0, 2, 2, 3, 3, 2, 2, 3, 1, 3, 4, 2, 3, 2, 3, 3, 2, 2, 5, 0, 4, 3, 0, 0, 0, 0, 0, 3, 4, 4, 3, 0, 2, 2, 0, 2, 3, 2, 3, 3, 3, 3, 3, 2, 3, 2, 3, 0, 2, 5, 3, 3, 3, 5, 2, 2, 1, 3, 2, 1, 3, 1, 1, 3, 2, 5, 3, 2, 3, 3, 2, 3, 3, 5, 3, 0, 3, 0, 3, 1, 5, 4, 3, 2, 2, 2, 5, 3, 0, 4, 5, 0, 3, 2, 3, 3, 3, 4, 5, 4, 5, 5, 3, 3, 4, 2, 2, 3, 3, 0, 0, 2, 2, 0, 3, 2, 2, 3, 3, 5, 5, 0, 5, 3, 3, 2, 3, 3, 2]
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [38]:
# Build the model with tensorflow.keras.
# The general idea is to reduce the size by maxpooling and 
# extract more features with convolutions of an increasing 
# number of filters.
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 5, padding='same', activation='relu', 
                        input_shape=(image_shape[0], image_shape[1], 1)),  # learn why the 1 is required...
    tf.keras.layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, 5, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((4, 4), (4, 4), padding='same'),
    tf.keras.layers.Flatten(),
    # keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(num_TICI_classes, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 1024, 1024, 32)    832       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_v2_1 (Ba (None, 256, 256, 32)      128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 64)      51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 262144)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 262144)           

In [39]:
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

In [40]:
# Add one dimension to the images input for channels.
images_as_model_input = np.array(images).reshape(nfiles, image_shape[0], image_shape[1], 1)
print(images_as_model_input.shape)    

(201, 1024, 1024, 1)


In [47]:
kf = KFold(n_splits=4)
for train_index, test_index in kf.split(images_as_model_input):
    x_train, x_val = images_as_model_input[train_index], images_as_model_input[test_index]
    y_train, y_val = TICI_one_hot[train_index], TICI_one_hot[test_index]
    
    model.fit(
        x_train,
        y_train,
        batch_size=10, ##################### TODO
        epochs=3, ##################### TODO
        verbose=1, # progress bar
        validation_data=(x_val, y_val)
    )

Train on 150 samples, validate on 51 samples
Epoch 1/3
150/150 [==============================] - 77s 513ms/sample - loss: 0.8794 - accuracy: 0.7000 - val_loss: 13.4991 - val_accuracy: 0.1176
Epoch 2/3
150/150 [==============================] - 75s 499ms/sample - loss: 0.7298 - accuracy: 0.7267 - val_loss: 12.5254 - val_accuracy: 0.1176
Epoch 3/3
150/150 [==============================] - 73s 486ms/sample - loss: 0.5809 - accuracy: 0.8133 - val_loss: 13.5854 - val_accuracy: 0.1176
Train on 151 samples, validate on 50 samples
Epoch 1/3
151/151 [==============================] - 72s 479ms/sample - loss: 0.6536 - accuracy: 0.7550 - val_loss: 18.9371 - val_accuracy: 0.0200
Epoch 2/3
151/151 [==============================] - 77s 511ms/sample - loss: 0.4738 - accuracy: 0.8477 - val_loss: 18.3682 - val_accuracy: 0.1200
Epoch 3/3
151/151 [==============================] - 75s 498ms/sample - loss: 0.4750 - accuracy: 0.8212 - val_loss: 19.4057 - val_accuracy: 0.3000
Train on 151 samples, valida